# **Tutorial 4 - Public opinion on the climate emergency & why it matters**
## **Text analysis of COP tweets**

**Week 2, Day 3: IPCC Socio-economic Basis**

**Content creators:** Maximilian Puelma Touzel

**Content reviewers:** Peter Ohue, Derick Temfack, Zahra Khodakaramimaghsoud, Peizhen Yang, Younkap Nina DuplexLaura Paccini, Sloane Garelick, Abigail Bodner, Manisha Sinha, Agustina Pesce, Dionessa Biton, Cheng Zhang, Jenna Pearson, Chi Zhang, Ohad Zivan

**Content editors:** Jenna Pearson, Chi Zhang, Ohad Zivan

**Production editors:** Wesley Banfield, Jenna Pearson, Chi Zhang, Ohad Zivan

**Our 2023 Sponsors:** NASA TOPS

# **Tutorial Objectives** 
In this tutorial, we will explore a dataset derived from Twitter, focusing on public sentiment surrounding the [Conference of Parties (COP) climate change conferences](https://unfccc.int/process/bodies/supreme-bodies/conference-of-the-parties-cop). We will use data from a published study by [Falkenberg et al. *Nature Clim. Chg.* 2022](https://www.nature.com/articles/s41558-022-01527-x). This dataset encompasses tweets mentioning the COP conferences, which bring together world governments, NGOs, and businesses to discuss and negotiate on climate change progress. Our main objective is to understand public sentiment about climate change and how it has evolved over time through an analysis of changing word usage on social media. In the process, we will also learn how to manage and analyze large quantities of text data.

The tutorial is divided into sections, where we first delve into loading and inspecting the data, examining the timing and languages of the tweets, and analyzing sentiments associated with specific words, including those indicating 'hypocrisy'. We'll also look at sentiments regarding institutions within these tweets and compare the sentiment of tweets containing 'hypocrisy'-related words versus those without. This analysis is supplemented with visualization techniques like word clouds and distribution plots.

By the end of this tutorial, you will have developed a nuanced understanding of how text analysis can be used to study public sentiment on climate change and other environmental issues, helping us to navigate the intricate and evolving landscape of climate communication and advocacy.


# **Setup**

Import libraries

In [6]:
!pip install afinn
!pip install wordcloud
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 11.5 MB/s eta 0:00:00


In [7]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#notebook config
from IPython.display import display, HTML
import datetime
import re
import nltk
from nltk.corpus import stopwords
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import urllib.request  # the lib that handles the url stuff
from afinn import Afinn
import pooch

In [ ]:
# @title Figure settings
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle")

In [ ]:
# @title Video 2: A Simple Greenhouse Model
#Tech team will add code to format and display the video

In [ ]:
sns.set_style("ticks", {'axes.grid' : False})
display(HTML("<style>.container { width:100% !important; }</style>"))

# **Section 1: Data preprocessing**

We have performed the following preprocessing steps for you (simply follow along; there is no need to execute any commands in this section):

Every Twitter messages (hereon called *tweets*) has an ID. IDs of all tweets mentioning `COPx` (x=20-26) used in Falkenberg et al. (2022) were placed by the authors in an [osf archive](https://osf.io/nu75j). Download the 7 .csv files (one for each COP) [here](https://osf.io/download/pr29x/) 

The `twarc2` program serves as an interface with the Twitter API, allowing users to retrieve full tweet content and metadata by providing the tweet ID. Similar to GitHub, you need to create a Twitter API account and configure `twarc` on your local machine by providing your account authentication keys. To rehydrate a set of tweets using their IDs, you can use the following command: `twarc2 hydrate source_file.txt store_file.jsonl`. In this command, each line of the `source_file.txt` represents a Twitter ID, and the hydrated tweets will be stored in the `store_file.jsonl`.


- First format the downloaded IDs and split them into separate files (*batches*) to make hydration calls to the API more time manageable (hours versus days - this is slow because of an API-imposed limit of 100 tweets/min.). 

In [ ]:
# import os
# dir_name='Falkenberg2022_data/'
# if not os.path.exists(dir_name):
#     os.mkdir(dir_name)
# batch_size = int(1e5)
# download_pathname=''#~/projects/ClimateMatch/SocioEconDay/Polarization/COP_Twitter_IDs/
# for copid in range(20,27):
#     df_tweetids=pd.read_csv(download_pathname+'tweet_ids_cop'+str(copid)+'.csv')    
#     for batch_id,break_id in enumerate(range(0,len(df_tweetids),batch_size)):
#         file_name="tweetids_COP"+str(copid)+"_b"+str(batch_id)+".txt"
#         df_tweetids.loc[break_id:break_id+batch_size,'id'].to_csv(dir_name+file_name,index=False,header=False)

- Make the hydration calls for COP26 (this took 4 days to download 50GB of data for COP26).

In [ ]:
# import glob
# import time
# copid=26
# filename_list = glob.glob('Falkenberg2022_data/'+"tweetids_COP"+str(copid)+"*")
# dir_name='tweet_data/'
# if not os.path.exists(dir_name):
#     os.mkdir(dir_name)
# file_name="tweetids_COP"+str(copid)+"_b"+str(batch_id)+".txt"
# for itt,tweet_id_batch_filename in enumerate(filename_list):
#     strvars=tweet_id_batch_filename.split('/')[1].split('.')[0].split('_')
#     tweet_store_filename = dir_name+'tweets_'+strvars[1]+'_'+strvars[2]+'.json'
#     if not os.path.exists(tweet_store_filename):
#         st=time.time()
#         os.system('twarc2 hydrate '+tweet_id_batch_filename+' '+tweet_store_filename)
#         print(str(itt)+' '+str(strvars[2])+" "+str(time.time()-st))

- Load data

Inspect and pick a chunk size. Note, by default, there are 100 tweets per line in the `.json` files returned by the API. Given we asked for 1e5 tweets/batch, there should be 1e3 lines in these files.

In [ ]:
# copid=26
# batch_id = 0
# tweet_store_filename = 'tweet_data/tweets_COP'+str(copid)+'_b'+str(batch_id)+'.json'
# num_lines = sum(1 for line in open(tweet_store_filename))
# num_lines

Now we read in the data, iterating over chunks in each batch and only store the needed data in a dataframe (takes 10-20 minutes to run). Let's look at when the tweets were posted, what language they are in, and the tweet text:

In [ ]:
# selected_columns = ['created_at','lang','text']

In [ ]:
# st=time.time()
# filename_list = glob.glob('tweet_data/'+"tweets_COP"+str(copid)+"*")
# df=[]
# for tweet_batch_filename in filename_list[:-1]:
#     reader = pd.read_json(tweet_batch_filename, lines=True,chunksize=1)
# #     df.append(pd.DataFrame([item[selected_columns] for sublist in reader.data.values.tolist()[:-1] for item in sublist] )[selected_columns])
#     dfs=[]
#     for chunk in reader:
#         if 'data' in chunk.columns:
#             dfs.append(pd.DataFrame(list(chunk.data.values)[0])[selected_columns])
#     df.append(pd.concat(dfs,ignore_index=True))
# #     df.append(pd.DataFrame(list(reader.data)[0])[selected_columns])
# df=pd.concat(df,ignore_index=True)
# df.created_at=pd.to_datetime(df.created_at)
# print(str(len(df))+' tweets took '+str(time.time()-st))
# df.head()

- finally, store the data in the efficiently compressed feather format

In [ ]:
# df.to_feather('stored_tweets') 

(run notebook from here)

# **Section 2: Load and inspect data**

Load the preprocessed data

In [ ]:
path = pooch.retrieve("https://osf.io/download/8p52x/",known_hash=None)
df=pd.read_feather(path) #takes a couple minutes to download

Let's check the timing of the tweets relative to the COP26 event (duration shaded in blue) to see how the tweets are spread over time.

In [ ]:
total_tweetCounts=df.created_at.groupby(df.created_at.apply(lambda x: x.date)).count().rename('counts')
total_tweetCounts.reset_index().plot(x='created_at',y='counts',figsize=(20,5),style='.-') 
pl.xticks(rotation=45,ha='right');
pl.gca().set_yscale('log')
COPdates=[datetime.datetime(2021,10,31), datetime.datetime(2021,11,12)] #shade the duration of the COP26 to guide the eye
pl.gca().axvspan(*COPdates, alpha=0.3); #gray region

Who is tweeting about this COP? Look at how many tweets were posted in various languages:

In [ ]:
counts=df.lang.value_counts().reset_index()

The stored language name of the tweet is stored as a code name. Pull a language code dictionary from the web and use it to translate the language code to the language name.

In [ ]:
target_url='https://gist.githubusercontent.com/carlopires/1262033/raw/c52ef0f7ce4f58108619508308372edd8d0bd518/gistfile1.txt'
exec(urllib.request.urlopen(target_url).read())
lang_code_dict=dict(iso_639_choices)
counts=counts.replace({'index':lang_code_dict})
counts

### **Coding Exercise 2.1**
Find your native language code in the dictionary printed below and use it to select the COP tweets that were written in your language!

In [ ]:
lang_code_dict

Select tweets of a given language

In [ ]:
language_code = 'en'
df=df.loc[df.lang==language_code,:].reset_index(drop=True)
pd.options.display.max_rows=100 #see up to 100 entries
pd.options.display.max_colwidth=250 #widen how much text is presented of each tweet
df.sample(100)

# **Section 3: Word set prevalence**

[Falkenberg et al.](https://www.nature.com/articles/s41558-022-01533-z) investigated the hypothesis that *public sentiment* around the COP conferences has increasingly framed them as hypocritical ("political hypocrisy as a topic of cross-ideological appeal"). The authors operationalized hypocrisy language as any tweet containing any of the following words:

In [ ]:
selected_words = ['hypocrisy', 'hypocrite', 'hypocritical', 'greenwash','green wash', 'blah'] #the last 3 words don't add much. Greta Thurnberg's 'blah, blah blah' speech on Sept. 28th 2021.

### **Questions 3.1**
How might this matching procedure be limited in its ability to capture this sentiment?

In [ ]:
# to_remove explanation

"""
Our approach is based on a predetermined dictionary, which might have several limitations in accurately capturing sentiment. Contextual ignorance (e.g., the word "not" can reverse the sentiment of the following word, but this isn't captured in a simple matching procedure) and language and cultural differences cannot be well captured.
"""

The authors then searched for these words within a distinct dataset across all COP conferences (this dataset was not made openly accessible; result: the figure using that data is here: https://www.nature.com/articles/s41558-022-01527-x/figures/7). They found that hypocrisy has been mentioned more in recent COP conferences.


Here, we will instead focus on their acessible COP26 dataset and assess the nature of comments about specific topics, like political hypocrisy.

In [ ]:
selectwords_detector = re.compile(r'\b(?:{0})\b'.format('|'.join(selected_words))) #to make a word detector for a wordlist faster to run, compile it!
df['select_talk']=df.text.apply(lambda x: selectwords_detector.search(x,re.IGNORECASE)) # look through whole dataset, flagging tweets with select_talk (computes in under a minute)

Let's pull these tweets out and look at thier occurence statistics relative to those of the whole dataset computed above

In [ ]:
selected_tweets=df.loc[~df.select_talk.isnull(),:]
selected_tweet_counts=selected_tweets.created_at.groupby(selected_tweets.created_at.apply(lambda x: x.date)).count().rename('counts')
selected_tweet_fraction=selected_tweet_counts/total_tweetCounts

In [ ]:
selected_tweet_fraction.reset_index().plot(x='created_at',y='counts',figsize=(20,5),style=['.-']) #this doesn't 
pl.xticks(rotation=45,ha='right');
pl.gca().axvspan(*COPdates, alpha=0.3) #gray region
pl.gca().set_ylabel('fraction talking about hypocrisy')

Note these are normalized so larger fractions closer to the COP26 dates correspond to significantly more absolute number of tweets. 

Let's look at what these tweets say. Take 100 randomly sampled.

In [ ]:
selected_tweets.text.sample(50).values

*Exercise*: Pick another topic, propose a list of topic words, and run the same analysis. (Example topic: "renewable technology")

In [ ]:
#copy code here 

## Section 3: Sentiment analysis

The hypocrisy displayed above seems charged against international organizations central to globalization (e.g. G7). Let's assess that hypothesis by analyzing the sentiment of the subset of these hypocrisy tweets that mention these institutions. 

(part of the computation flow in what follows was taken from Caren Neal's tutorial: https://nealcaren.org/lessons/wordlists/)

We'll match words in tweets to sentiment scores using a dictionary method. The particular dictionary we will use is compiled in the AFINN package and reflects a scoring between -5 and 5 in which words were rated for valence. Let's initialize it for the language we've chosen.

In [ ]:
afinn = Afinn(language=language_code)

Load the dictionary

In [ ]:
afinn_wl_url = ('https://raw.githubusercontent.com'
                '/fnielsen/afinn/master/afinn/data/AFINN-111.txt')
afinn_wl_df = pd.read_csv(afinn_wl_url,
                          header=None, # no column names
                          sep='\t',  # tab sepeated
                          names=['term', 'value']) #new column names
seed = 808 # seed for sample so results are stable
afinn_wl_df.sample(10, random_state = seed)

let's look at the distribution of scores over all words in the dictionary

In [ ]:
afinn_wl_df.value.value_counts().sort_index().plot.bar()
pl.gca().set_xlabel('Finn score')
pl.gca().set_ylabel('dictionary counts')

These scores were assigned to words based on labeled tweets ([validation paper](http://www2.imm.dtu.dk/pubdb/edoc/imm6006.pdf)).  

Before focussing on sentiments about institutions within the hypocrisy tweets, let's look at the hypocrisy tweets in comparison to non-hypocrisy tweets. This will take some more intensive computation, so let's only perform it on a 1% subsample of the dataset

In [ ]:
smalldf=df.sample(frac=0.01)

In [ ]:
smalldf['afinn_score'] = smalldf.text.apply(afinn.score) #intensive computation! We have reduced the data set to frac=0.01 it's size so it takes ~1 min. (the full dataset takes 1hrs 50 min.)

In [ ]:
smalldf['afinn_score'].describe()

The score is a sum over all dictionary words in the tweet so is larger for longer tweets. 

A rough approach to making scores comparable across tweets of different length is convert to a per-word score by normalizing each tweet's score by its word count (note this is not a per-dictionary word score and so this rough approach introduces a bias dependent on how much of each tweet is formed by dictionary words). Let's call this normalized score *afinn_adjusted*.

In [ ]:
def word_count(text_string):
    '''Calculate the number of words in a string'''
    return len(text_string.split())
smalldf['word_count']=smalldf.text.apply(word_count)
smalldf['afinn_adjusted'] = smalldf['afinn_score'] / smalldf['word_count'] #note this isn't a percentage

In [ ]:
smalldf['afinn_adjusted'].describe()

Now let's look at the sentiment of tweets with hypocrisy words versus those without those words. We'll first plot for some other possibly negative words for reference.

In [ ]:
for sel_words in [['Fossil'],['G7'],['Boris'],['Davos'],selected_words]:
    sel_name = sel_words[0] if len(sel_words)==1 else 'select_talk'
    selectwords_detector = re.compile(r'\b(?:{0})\b'.format('|'.join(sel_words))) #compile for speed!
    smalldf[sel_name] = smalldf.text.apply(lambda x: selectwords_detector.search(x,re.IGNORECASE) is not None) #flag if tweet has word(s)

make cumulative distribtion plots of score distributions

In [ ]:
for sel_words in [['Fossil'],['G7'],['Boris'],['Davos'],selected_words]:
    sel_name =sel_words[0] if len(sel_words)==1 else 'select_talk'
    fig,ax=pl.subplots()
    pl.gca().set_xlim(-1,1)
    pl.gca().set_xlabel('adjusted Finn score')
    pl.gca().set_ylabel('probabilty')
    counts,bins=np.histogram(smalldf.loc[smalldf[sel_name],'afinn_adjusted'],bins=np.linspace(-1,1,101),density=True)
    pl.plot(bins[:-1],np.cumsum(counts),color='C0',label=sel_name+' tweets')
    counts,bins=np.histogram(smalldf.loc[~smalldf[sel_name],'afinn_adjusted'],bins=np.linspace(-1,1,101),density=True)
    pl.plot(bins[:-1],np.cumsum(counts),color='C1',label='non-'+sel_name+' tweets')
    pl.gca().axvline(0,color=[0.7]*3,zorder=1)
    pl.legend()
    pl.title('cumulative Finn score distribution for '+sel_name+ ' occurence')

Observe that tweets containing the selected *hypocrisy*-associated words set are scored much more negative than all of these. So what is the content of these selected tweets?

## Section 4: word clouds

To analyze word usage, let's first vectorize the text data. Vectorization here means giving each word in the vocabulary an index and transforming each word sequence in each to its vector representation whose sequence of elements are the sequence of the corresponding word indices (e.g. the response `['I','love','icecream']` maps to something like `[34823,5937,79345]`). 

We'll use and compare two methods: term-frequency (tf) and term-frequency inverse document frequency (tfidf). In the next cell you can see their definition.

Let's write down what they compute by denoting the index, $d$, over the $D$ documents and the index, $w$, over the $W$ words in the vocabulary (the list of all the words found in all the tweets, which we'll call documents): 
- term frequency, $\mathrm{tf}(w,d)$. The frequency of a word $w$ in a document $d$ is $$\mathrm{tf}(w,d):=\frac{n(w,d)}{n(d)},$$ where $n(w,d)$ is the number of times term $w$ is in document $d$ and $n(d)=\sum_{w=1}^W n(w,d)$ is the total number of words in document $d$. The term frequency over all the documents is then, $$\mathrm{tf}(w):=\frac{\sum_{d=1}^D n(d)\mathrm{tf}(w,d)}{N},$$ where the denominator $N=\sum_{d=1}^D n(d)$ is just the total word count across all documents.
- term frequency-inverse document frequency, $\mathrm{Tfidf}(w,d):=\mathrm{tf}(w,d)\mathrm{idf}(w)$. Here, $$\mathrm{idf}(w)=\frac{\log(D+1)}{\log(n(w)+1)+1},$$ where $n(w)$ is the number of documents in which term $t$ appears, i.e. $n(w,d)>0$. Idf is like an inverse document frequency. The `sklearn` package then uses $$\mathrm{Tfidf}(w)=\frac{1}{D}\sum_{d=1}^D \frac{\mathrm{Tfidf}(w,d)}{||\mathrm{Tfidf}(\cdot,d)||},$$ where $||\vec{x}||=\sqrt{\sum_{i=1}^Nx^2_i}$ is the Euclidean norm.

$\mathrm{Tfidf}$ aims to add more discriminability to frequency as a word relevance metric by downweighting words that appear in many documents since these common words are less discriminative.

Let's run both these methods and store the vectorized data in a dictionary

In [ ]:
vectypes=['counts','Tfidf']
def vectorize(doc_data,ngram_range=(1,1),remove_words=[],min_doc_freq=1):

    vectorized_data_dict={}
    for vectorizer_type in vectypes:
        if vectorizer_type=='counts':
            vectorizer = CountVectorizer(stop_words=remove_words,min_df=min_doc_freq,ngram_range=ngram_range) 
        elif vectorizer_type=='Tfidf':
            vectorizer = TfidfVectorizer(stop_words=remove_words,min_df=min_doc_freq,ngram_range=ngram_range) 

        vectorized_doc_list = vectorizer.fit_transform(data).todense().tolist()
        feature_names = vectorizer.get_feature_names_out() # or  get_feature_names() depending on scikit learn version
        print('vocabulary size:'+str(len(feature_names)))
        wdf = pd.DataFrame(vectorized_doc_list, columns=feature_names)
        vectorized_data_dict[vectorizer_type]=wdf
    return vectorized_data_dict,feature_names

def plot_wordcloud_and_freqdist(wdf,title_str,feature_names):
    '''
    Plots a word cloud
    '''
    pixel_size=600
    x,y =np.ogrid[:pixel_size,:pixel_size]
    mask = (x -pixel_size/2)**2+(y-pixel_size/2)**2>(pixel_size/2-20)**2
    mask=255*mask.astype(int)
    wc=WordCloud(background_color="rgba(255, 255, 255, 0)", mode="RGBA", mask=mask, max_words=50)#,relative_scaling=1)
    wordfreqs=wdf.T.sum(axis=1)
    num_show=50
    sorted_ids=np.argsort(wordfreqs)[::-1]

    fig,ax=pl.subplots(figsize=(10,5))
    ax.bar(x=range(num_show),height=wordfreqs[sorted_ids][:num_show])
    ax.set_xticks(range(num_show))
    ax.set_xticklabels(feature_names[sorted_ids][:num_show],rotation=45,fontsize=8,ha='right');
    ax.set_ylabel('total frequency')
    ax.set_title(title_str+' vectorizer')
    ax.set_ylim(0,10*wordfreqs[sorted_ids][int(num_show/2)])
    
    ax_wc = inset_axes(ax,width='90%',height='90%')
    wc.generate_from_frequencies(wordfreqs)
    ax_wc.imshow(wc,interpolation='bilinear')
    ax_wc.axis('off')
    
nltk.download('stopwords') #downloads basic stop words, i.e. words with little semantic value  (e.g. "the"), to be used as words to be removed
remove_words = stopwords.words("english") 

Okay so let's vectorize and look at the wordclouds for single word statistics. Let's explicitly exclude some words and implicity exclude ones that appear in fewer than some threshold number of documents

In [ ]:
data=selected_tweets['text'].sample(frac=0.1).values #reduce size since the vectorization computation transforms the corpus into an array of large size (vocabulary size x number of tweets)
#let's add some more words that we don't want to track (you can generate this kind of list iteratively by looking at the results and adding to this list):
remove_words += ['cop26','http','https','30','000','je','rt','climate','limacop20','un_climatetalks','climatechange','via','ht','talks','unfccc','peru','peruvian','lima','co']
print(str(len(data))+" tweets")
min_doc_freq=5/len(data)

In [ ]:
ngram_range=(1,1) #start and end number of words
vectorized_data_dict,feature_names=vectorize(selected_tweets,ngram_range=ngram_range,remove_words=remove_words,min_doc_freq=min_doc_freq)
for vectorizer_type in vectypes:
    plot_wordcloud_and_freqdist(vectorized_data_dict[vectorizer_type],vectorizer_type,feature_names)

Note in the histograms how the TFidf vectorizer has scaled down the hypocrisy words such that they are less outlyingly prevalent relative to the count vectorizer. 

There are some words here  (e.g. `private` and `jet`) that look like they likely would appear in pairs. Let's tell the vectorizer to also look for high frequency *pairs* of words.

In [ ]:
ngram_range= (1,2)#start and end number of words
vectorized_data_dict,feature_names=vectorize(selected_tweets,ngram_range=ngram_range,remove_words=remove_words,min_doc_freq=min_doc_freq)
for vectorizer_type in vectypes:
    plot_wordcloud_and_freqdist(vectorized_data_dict[vectorizer_type],vectorizer_type,feature_names)

The hypocrisy words take up so much frequency it is hard to see what the remaining words are. To clear this list a bit more, let's also remove the hypocrisy words altogether.

In [ ]:
remove_words += selected_words

In [ ]:
ngram_range= (1,2)#start and end number of words
vectorized_data_dict,feature_names=vectorize(selected_tweets,ngram_range=ngram_range,remove_words=remove_words,min_doc_freq=min_doc_freq)
for vectorizer_type in vectypes:
    plot_wordcloud_and_freqdist(vectorized_data_dict[vectorizer_type],vectorizer_type,feature_names)

Observe terms that we might have expected are associated to hypocrisy, e.g. "flying". Even when allowing for pairs, the semantics are hard to extract from this analysis that ignores the correlations in usage among multiple words. 

To futher assess statistics, one approach is use a generative model with latent structure.

Topic models (the [structural topic model](https://www.structuraltopicmodel.com/) in particular) are a nice modelling framework to start analyzing those correlations.

For a modern introduction to text analysis in the social sciences, I recommend the textbook:

    Text as Data: A New Framework for Machine Learning and the Social Sciences (2022) by Justin Grimmer, Margaret E. Roberts, and Brandon M. Stewart

# **Summary**
In this tutorial, you've learned how to analyze large amounts of text data from social media to understand public sentiment about climate change. You've been introduced to the process of loading and examining Twitter data, specifically relating to the COP climate change conferences. You've also gained insights into identifying and analyzing sentiments associated with specific words, with a focus on those indicating 'hypocrisy'.

We used techniques to normalize sentiment scores and to compare sentiment among different categories of tweets. You have also learned about text vectorization methods, term-frequency (tf) and term-frequency inverse document frequency (tfidf), and their applications in word usage analysis. This tutorial provided you a valuable stepping stone to further delve into text analysis, which could help deeper our understanding of public sentiment on climate change. Such analysis helps us track how global perceptions and narratives about climate change evolve over time, which is crucial for policy planning and climate communication strategies.

This tutorial therefore not only provided you with valuable tools for text analysis but also demonstrated their potential in contributing to our understanding of climate change perceptions, a key factor in driving climate action.